# Summary Statistics

In [1]:
import sqlite3

import httpx
import pandas as pd

from litreview import ClinicalTrials

# Make pandas display numbers in non-scientific notation
pd.set_option("display.float_format", lambda x: f"{x:0.1f}")

## Retrieve Integer Fields

In [2]:
API_BASE = "https://clinicaltrials.gov/api/v2"
API_FIELD_VALUES = API_BASE + "/stats/field/values"

response = httpx.get(API_FIELD_VALUES, params={"types": "INTEGER|NUMBER"})
response.raise_for_status()
data = response.json()
integer_fields = [field["piece"] for field in data]
integer_fields

['DispFirstPostYear',
 'DispFirstSubmitYear',
 'EnrollmentCount',
 'EventGroupDeathsNumAffected',
 'EventGroupDeathsNumAtRisk',
 'EventGroupOtherNumAffected',
 'EventGroupOtherNumAtRisk',
 'EventGroupSeriousNumAffected',
 'EventGroupSeriousNumAtRisk',
 'LargeDocSize',
 'LastUpdatePostYear',
 'LastUpdateSubmitYear',
 'NumArmGroupInterventionNames',
 'NumArmGroups',
 'NumAvailIPDs',
 'NumBaselineCategories',
 'NumBaselineClasses',
 'NumBaselineDenoms',
 'NumBaselineGroups',
 'NumBaselineMeasureDenoms',
 'NumBaselineMeasurements',
 'NumBaselineMeasures',
 'NumCentralContacts',
 'NumCollaborators',
 'NumCollaboratorsPlusLead',
 'NumConditionAncestors',
 'NumConditionBrowseBranches',
 'NumConditionBrowseLeafs',
 'NumConditionMeshes',
 'NumConditions',
 'NumDesignWhoMaskeds',
 'NumEventGroups',
 'NumEvents',
 'NumFlowAchievements',
 'NumFlowDropWithdraws',
 'NumFlowGroups',
 'NumFlowMilestones',
 'NumFlowPeriods',
 'NumFlowReasons',
 'NumIPDSharingInfoTypes',
 'NumInterventionAncestors',
 'N

## Load Data from Database

In [3]:
db_connection = sqlite3.connect("../clinical_trials.db")
trials = ClinicalTrials(connection=db_connection, schema_directory="../files/schema.json")
trials.query(*integer_fields)

KeyError: 'DispFirstPostYear'